**Encriptación usando las funciones Chr y Ord**

In [49]:
def encriptacion(string, step):
  output = ""
  for caracter in string:
    #cada caracter tiene asignado un valor numerico
    output += chr(ord(caracter)-step) #si cambiamos el valor de este numero también cambiará el caracter 

  return output

def desencriptacion(string, step):
  output = ""
  for caracter in string:
    #para la desencriptación simplemente se hace lo contrario, volviendo al valor del caracter original
    output += chr(ord(caracter)+step)

  return output


if __name__ == "__main__":
  string = input('')
  step = 5
  encriptado = encriptacion(string, step)
  desencriptado = desencriptacion(encriptado, step)
  print(f"Original: {string}")
  print(f"Encriptado: {encriptado}")
  print(f"Desencriptado: {desencriptado}")

hola mundo
Original: hola mundo
Encriptado: cjg\hpi_j
Desencriptado: hola mundo


**Estadísticas actuales del covid a nivel mundial**

In [ ]:
!pip install chart_studio -q

In [57]:
import pandas as pd
import numpy as np
import plotly.express as px
import chart_studio

#importamos los datos de Novel Coronavirus COVID-19 Data Repository by Johns Hopkins CSSE
cd = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
dd = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
rd = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
#nos deshacemos de datos innesecarios
confirmados = cd.drop(columns=['Lat', 'Long','Province/State'])
muertos = dd.drop(columns=['Lat', 'Long','Province/State'])
recuperados = rd.drop(columns=['Lat', 'Long','Province/State'])
#el numero de casos activos se calcula a partir del número de personas confirmadas - muertos - recuperados
active =confirmados.copy()
active.iloc[:,1:] = active.iloc[:,1:] - muertos.iloc[:,1:] - recuperados.iloc[:,1:]


In [58]:
#establecemos los datos necesarios
CONFIRMADOS = confirmados.groupby(by='Country/Region').aggregate(np.sum).T
CONFIRMADOS.index.name = 'date'
CONFIRMADOS =  CONFIRMADOS.reset_index()

RECUPERADOS = recuperados.groupby(by='Country/Region').aggregate(np.sum).T
RECUPERADOS.index.name = 'date'
RECUPERADOS =  RECUPERADOS.reset_index()

ACTIVOS = active.groupby(by='Country/Region').aggregate(np.sum).T
ACTIVOS.index.name = 'date'
ACTIVOS =  ACTIVOS.reset_index()

MUERTOS = muertos.groupby(by='Country/Region').aggregate(np.sum).T
MUERTOS.index.name = 'date'
MUERTOS =  MUERTOS.reset_index()

In [60]:
# Numero de Casos confirmados por dia en el mundo
column_names = ["Fecha", "Confirmados", "Recuperados","Muertos"]
world = pd.DataFrame(columns = column_names)
world['Fecha'] = CONFIRMADOS['date'].copy()
world['Confirmados'] = CONFIRMADOS.iloc[:,1:].sum(1)
world['Muertos'] = MUERTOS.iloc[:,1:].sum(1)
world['Recuperados'] = RECUPERADOS.iloc[:,1:].sum(1)
world['Activos'] = ACTIVOS.iloc[:,1:].sum(1)
world

,Fecha,Confirmados,Recuperados,Muertos,Activos
0,1/22/20,557,30,17,510.0
1,1/23/20,657,32,18,605.0
2,1/24/20,944,39,26,877.0
3,1/25/20,1437,42,42,1351.0
4,1/26/20,2120,56,56,2006.0
...,...,...,...,...,...
1040,11/27/22,641599225,0,6630837,596628074.0
1041,11/28/22,642023968,0,6631992,597049958.0
1042,11/29/22,642748672,0,6633444,597772405.0
1043,11/30/22,643273774,0,6635022,598294049.0


In [61]:
#histograma donde cada columna es una variable
data = pd.DataFrame(world.iloc[-1,:]).T
wrld = data.melt(id_vars="Fecha", value_vars=[ "Confirmados","Activos","Recuperados","Muertos"])
fig = px.bar(wrld, x="variable" , y="value", color= 'variable', text='value',
             color_discrete_sequence=["blue","crimson","green", "black"],
             height=500, width=600,
             title= f'Total de Casos Mundiales de COVID 19')

fig.update_traces(textposition='outside')#función para posicionar los valores de las barras por fuera de estas
fig.layout.update(showlegend=False, yaxis =  {"title": {"text": "Numero de Personas"}}, xaxis =  {"title": {"text": ""}})


fig.show()